In [1]:
from operator import and_, or_
from functools import reduce

import pathlib as pl
import pandas as pd
import numpy as np

from tinydb import TinyDB

from candas.test import QuantStudio
from candas.lims import Librarian, library
from candas.lims.schema import Fragment, Amplicon, Core, Primer, ProbeOligo, ProbeSeq

In [2]:
def list_and(lst: list) -> str:
    if len(lst) == 1:
        msg = f'{lst[0]}'
    elif len(lst) == 2:
        msg = f'{lst[0]} and {lst[1]}'
    elif len(lst) > 2:
        msg = f'{", ".join(lst[:-1])}, and {lst[-1]}'
    else:
        msg = ''
    return msg

In [3]:
lib = Librarian().load(library).open()
# lib.get_by_name(Experiment, 'JG073A').wells[0].reaction

In [4]:
here = pl.Path.cwd()
data_pth = here / "raw_data"

In [5]:
data = pd.DataFrame()
meta = pd.DataFrame()
figs = pd.DataFrame()
smix = pd.DataFrame()

fragment_sequences = {}
fragment_descriptions = {}
primer_sequences = {}

In [6]:
seq_aliases = {
    "ISO": "S044.12",
    "BP50_GC25": "S044.5",
    "BP50_GC60": "S044.6",
    "BP50_GC75": "S044.7",
    "BP280_GC30": "S044.26",
    "BP280_GC70": "S044.27",
    "GC15": "S044.9",
    "GC25": "S044.10",
    "GC35": "S044.11",
    "GC45": "S044.12",
    "GC55": "S044.14",
    "GC65": "S044.15",
    "GC75": "S044.16",
    "GC80": "S044.17",
    "GC85": "S044.18",
    "BP240": "S044.25",
    "BP200": "S044.24",
    "BP160": "S044.21",
    "BP55": "S044.8",
    "BP40": "S044.4",
    "BP30": "S044.2",
    "BP15": "S044.0",
}

In [7]:
def name_reactions(df):
    return df.assign(
        Reaction=lambda df: df.apply(
            lambda row: f"{row.Experiment}_{row.Well:03d}", axis=1
        )
    )


def process_experiment(experiment, filename, cmax=50):
    qs = (
        QuantStudio(data_pth / filename, experiment)
        .import_data()
        .format_reactions()
        .index_reactions()
        .subtract_background()
        .normalize_reactions(cmax=cmax)
        .extract_endpoints(cmax=cmax, name="FAM-HEX")
    )
    wide = name_reactions(qs.reactions.wide)
    wide["Endpoint"] = wide.apply(lambda row: row.Fluorescence[cmax - 1], axis=1)
    return wide[
        [
            "Reaction",
            "Experiment",
            "Well",
            "Target",
            "Copies",
            "Reporter",
            "Sample",
            "CT",
            "Endpoint",
            "Cycle",
            "Fluorescence",
            "Corr_Fluorescence",
        ]
    ]


def parse_rxn_specs(filename, id_column="Sample"):
    db_file = data_pth / filename
    rxn_db = TinyDB(db_file)
    rxns = pd.DataFrame(
        [rxn["oligos"] | {id_column: rxn["name"]} for rxn in rxn_db.all()]
    )
    rxns = rxns.melt(
        id_vars=[id_column], var_name="component", value_name="concentration"
    ).assign(units="copies")
    return rxns


def extract_meta(data):
    meta = (
        data[["Reaction", "Target", "Copies"]]
        .rename(columns={"Target": "component", "Copies": "concentration"})
        .assign(units="copies", component=lambda df: df.component.replace(seq_aliases))
    )
    return neaten_meta(meta)


def neaten_meta(meta):
    return meta.drop_duplicates().reset_index(drop=True)[
        ["Reaction", "component", "concentration", "units"]
    ]


def append_data(new_data):
    assert new_data.shape[0] > 0
    new = (
        pd.concat(
            [
                data,
                new_data[
                    [
                        "Reaction",
                        "Experiment",
                        "Well",
                        "Reporter",
                        "CT",
                        "Endpoint",
                        "Cycle",
                        "Fluorescence",
                        "Corr_Fluorescence",
                    ]
                ],
            ],
            axis=0,
            ignore_index=True,
        )
        .drop_duplicates(subset=["Reaction", "Experiment", "Well", "Reporter"])
        .reset_index(drop=True)
    )

    print(f"The shape of `data` is now {new.shape[0]} x {new.shape[1]}")
    return new


def append_meta(new_meta, supermix=None):
    assert new_meta.shape[0] > 0

    if supermix is not None:
        assert isinstance(supermix, str)
        if not supermix.endswith("_supermix"):
            supermix = f"{supermix}_supermix"

        new_meta = pd.concat(
            [
                new_meta,
                pd.DataFrame(new_meta.Reaction.unique(), columns=["Reaction"]).assign(
                    component=supermix,
                    concentration=1,
                    units="X",
                ),
            ]
        )
        
        if supermix in smix.SuperMix.unique():
            smix_components = smix.query('SuperMix == @supermix').component.unique()
            new_meta = new_meta[~new_meta.component.isin(smix_components)]

    new = (
        pd.concat(
            [
                meta,
                new_meta[["Reaction", "component", "concentration", "units"]],
            ],
            axis=0,
            ignore_index=True,
        )
        .drop_duplicates()
        .replace({"component": seq_aliases})
        .reset_index(drop=True)
    )

    print(f"The shape of `meta` is now {new.shape[0]} x {new.shape[1]}")
    return new


def append_smix(new_smix, name):
    assert new_smix.shape[0] > 0
    if not new_smix.component.str.contains("Master Mix").any():
        new_smix.loc[-1] = ["TaqMan Fast Advanced Master Mix", 1, "X"]
    new_smix.reset_index(drop=True, inplace=True)

    assert isinstance(name, str)
    if not name.endswith("_supermix"):
        name = f"{name}_supermix"

    new = (
        pd.concat(
            [
                smix,
                new_smix.assign(SuperMix=name)[
                    ["SuperMix", "component", "concentration", "units"]
                ],
            ],
            axis=0,
            ignore_index=True,
        )
        .drop_duplicates()
        .reset_index(drop=True)
    )

    print(f"The shape of `smix` is now {new.shape[0]} x {new.shape[1]}")
    return new


def append_figs(reactions, alias, supermix=None):
    if supermix is None:
        supermix = alias
        
    assert isinstance(supermix, str)
    if not supermix.endswith("_supermix"):
        supermix = f"{supermix}_supermix"
        
    new_figs = pd.DataFrame(reactions, columns=["Reaction"]).assign(figure_alias=alias, supermix=supermix)
    new = (
        pd.concat([figs, new_figs], axis=0, ignore_index=True)
        .drop_duplicates()
        .reset_index(drop=True)
    )

    print(f"The shape of `figs` is now {new.shape[0]} x {new.shape[1]}")
    return new


def filter_reactions(this, this_filter):

    these_rxns = this.groupby("Reaction").filter(this_filter).Reaction.unique()

    n_rxns = len(these_rxns)
    assert n_rxns > 0
    print(n_rxns, "matching reactions")

    return these_rxns

# Fig 2 glass_rattlesnake

In [8]:
primers_045 = [
    ["FP004", 125, "nM"],
    ["RP004", 125, "nM"],
]

glass_rattlesnake_smix = pd.DataFrame(
    [["EvaGreen", 1, "X"]] + primers_045,
    columns=["component", "concentration", "units"],
)
smix = append_smix(glass_rattlesnake_smix, "glass_rattlesnake")

The shape of `smix` is now 4 x 4


In [9]:
JG045A_data = process_experiment("JG045A", "JG045A TMCC1 Gen3 EvaGreen.xlsx")

JG045A_targets = ["BP50_GC25", "BP50_GC75", "BP280_GC30", "BP280_GC70"]
JG045A_rates = [0.98, 0.67, 0.70, 0.49]
JG045A_outliers = [17,  41,  65,  89, 113, 137, 157, 158, 161, 320, 350, 373]

JG045A_data = JG045A_data.query(
    "Target in @JG045A_targets and Well not in @JG045A_outliers"
)

# JG034_data = JG034_data[JG034_data.Target.str.contains("GC55")].assign(Target="S044.13")

JG045A_meta = pd.DataFrame()
JG045A_meta["Reaction"] = JG045A_data.Reaction
JG045A_meta["component"] = JG045A_data.Target.map(seq_aliases)
JG045A_meta["concentration"] = JG045A_data.Copies
JG045A_meta["units"] = "copies"
JG045A_meta = neaten_meta(JG045A_meta)
JG045A_rxns = JG045A_meta.Reaction.unique()

data = append_data(JG045A_data)
meta = append_meta(JG045A_meta, supermix="glass_rattlesnake")
figs = append_figs(JG045A_rxns, "glass_rattlesnake", supermix="glass_rattlesnake")

The shape of `data` is now 44 x 9
The shape of `meta` is now 88 x 4
The shape of `figs` is now 44 x 3


# Fig 3 wool_wren

## JG034: Bipartite

In [10]:
primers_034 = [
    ["FP004", 125, "nM"],
    ["RP004", 125, "nM"],
]

probes_034 = [
    ["Pr002", 250, "nM"],
    ["Pr003", 250, "nM"],
]

plastic_yak_smix = pd.DataFrame(
    [[seq_aliases["GC55"], 10**5, "copies"]] + probes_034 + primers_034,
    columns=["component", "concentration", "units"],
)
smix = append_smix(plastic_yak_smix, "plastic_yak")

The shape of `smix` is now 10 x 4


In [11]:
JG034_data = process_experiment("JG034", "JG034 TMCC1 Gen2 Competitors - 59C v3.xlsx")

JG034_data = JG034_data[JG034_data.Target.str.contains("GC55")].assign(Target="S044.13")

JG034_meta = pd.DataFrame()
JG034_meta["Reaction"] = JG034_data.Reaction
JG034_meta["component"] = JG034_data.Target.str.split(" - ", expand=True)[0]
JG034_meta["concentration"] = JG034_data.Copies
JG034_meta["units"] = "copies"
JG034_meta = neaten_meta(JG034_meta)
JG034_rxns = JG034_meta.Reaction.unique()

data = append_data(JG034_data)
meta = append_meta(JG034_meta, supermix="plastic_yak")
figs = append_figs(JG034_rxns, "plastic_yak", supermix="plastic_yak")
figs = append_figs(JG034_rxns, "ceramic_zebu", supermix="plastic_yak")

The shape of `data` is now 72 x 9
The shape of `meta` is now 116 x 4
The shape of `figs` is now 58 x 3
The shape of `figs` is now 72 x 3


## JG071B: Tripartite

In [12]:
primers_071b = [
    ["FP001", 100, "nM"],
    ["RP001", 100, "nM"],
    ["FP005", 100, "nM"],
    ["RP005", 100, "nM"],
]

probes_071b = [
    ["Pr002", 200, "nM"],
    ["Pr003", 200, "nM"],
]

metal_anchovy_smix = pd.DataFrame(
    [["S057.3.2", 10**6, "copies"], ["S057.4.2", 10**4, "copies"]]
    + probes_071b
    + primers_071b,
    columns=["component", "concentration", "units"],
)
smix = append_smix(metal_anchovy_smix, "metal_anchovy")

The shape of `smix` is now 19 x 4


In [13]:
JG071B_data = process_experiment("JG071B", "JG071B Tripartite sweep GBP6.xlsx")

JG071B_meta = parse_rxn_specs("JG071B Reaction Specifications.json")
JG071B_meta = neaten_meta(JG071B_meta.merge(JG071B_data[["Reaction", "Sample"]]))


def this_filter(grp):
    conditions = [
        (grp.component == "S036.5") & (grp.concentration > 10),
        (grp.component == "S057.3.2") & (grp.concentration == 10**6),
        (grp.component == "S057.4.2") & (grp.concentration == 10**4),
    ]

    return reduce(and_, [cond.any() for cond in conditions])


JG071B_rxns = filter_reactions(JG071B_meta, this_filter)
JG071B_data = JG071B_data[JG071B_data.Reaction.isin(JG071B_rxns)]
JG071B_meta = JG071B_meta[JG071B_meta.Reaction.isin(JG071B_rxns)]
JG071B_meta = JG071B_meta[JG071B_meta.component == "S036.5"]

data = append_data(JG071B_data)
meta = append_meta(JG071B_meta, supermix="metal_anchovy")
figs = append_figs(JG071B_rxns, "metal_anchovy", supermix="metal_anchovy")
figs = append_figs(JG071B_rxns, "digital_bear", supermix="metal_anchovy")

13 matching reactions
The shape of `data` is now 98 x 9
The shape of `meta` is now 142 x 4
The shape of `figs` is now 85 x 3
The shape of `figs` is now 98 x 3


## JG074C: Antiparallel Redundant Module

In [14]:
primers_074C = [
    ["FP001", 100, "nM"],
    ["RP001", 100, "nM"],
    ["FP002", 100, "nM"],
    ["RP002", 100, "nM"],
]

probes_074C = [
    ["Pr001", 200, "nM"],
    ["Pr004", 200, "nM"],
    ["Pr007", 200, "nM"],
    ["Pr008", 200, "nM"],
]

electric_chinchilla_smix = pd.DataFrame(
    [["S037.01.01b", 10**6, "copies"], ["S036.1b", 10**4, "copies"]]
    + probes_074C
    + primers_074C,
    columns=["component", "concentration", "units"],
)
smix = append_smix(electric_chinchilla_smix, "electric_chinchilla")

The shape of `smix` is now 30 x 4


In [15]:
JG074C_data = process_experiment("JG074C", "JG074C_L2_500_competitor_sweep.xlsx")

JG074C_meta = parse_rxn_specs("JG074C Reaction Specifications.json")
JG074C_meta = neaten_meta(JG074C_meta.merge(JG074C_data[["Reaction", "Sample"]]))


def this_filter(grp):
    conditions = [
        (grp.component == "S037.01.01b") & (grp.concentration == 10**6),
        (grp.component == "S036.1b") & (grp.concentration == 10**2),
    ]

    return reduce(and_, [cond.any() for cond in conditions])


JG074C_rxns = filter_reactions(JG074C_meta, this_filter)
JG074C_data = JG074C_data[JG074C_data.Reaction.isin(JG074C_rxns)]
JG074C_meta = JG074C_meta[JG074C_meta.Reaction.isin(JG074C_rxns)]
JG074C_meta = JG074C_meta[JG074C_meta.component == "L2_500"]

data = append_data(JG074C_data)
meta = append_meta(JG074C_meta, supermix="electric_chinchilla")
figs = append_figs(JG074C_rxns, "electric_chinchilla", supermix="electric_chinchilla")
figs = append_figs(JG074C_rxns, "solar_dog", supermix="electric_chinchilla")

16 matching reactions
The shape of `data` is now 130 x 9
The shape of `meta` is now 174 x 4
The shape of `figs` is now 114 x 3
The shape of `figs` is now 130 x 3


## JG074D: Parallel Redundant Module

In [16]:
primers_074D = [
    ["FP001", 100, "nM"],
    ["RP001", 100, "nM"],
    ["FP002", 100, "nM"],
    ["RP002", 100, "nM"],
]

probes_074D = [
    ["Pr001", 200, "nM"],
    ["Pr004", 200, "nM"],
    ["Pr007", 200, "nM"],
    ["Pr008", 200, "nM"],
]

lunar_earwig_smix = pd.DataFrame(
    [["S037.01.01b", 10**8, "copies"], ["S036.0", 10**2, "copies"]]
    + probes_074D
    + primers_074D,
    columns=["component", "concentration", "units"],
)
smix = append_smix(lunar_earwig_smix, "lunar_earwig")

The shape of `smix` is now 41 x 4


In [17]:
JG074D_data = process_experiment("JG074D", "JG074D_L2_500_like_competitor_sweep.xlsx")

JG074D_meta = parse_rxn_specs("JG074D Reaction Specifications.json")
JG074D_meta = neaten_meta(JG074D_meta.merge(JG074D_data[["Reaction", "Sample"]]))


def this_filter(grp):
    conditions = [
        (grp.component == "S037.01.01b") & (grp.concentration == 10**8),
        (grp.component == "S036.0") & (grp.concentration == 10**2),
    ]

    return reduce(and_, [cond.any() for cond in conditions])


JG074D_rxns = filter_reactions(JG074D_meta, this_filter)
JG074D_data = JG074D_data[JG074D_data.Reaction.isin(JG074D_rxns)]
JG074D_meta = JG074D_meta[JG074D_meta.Reaction.isin(JG074D_rxns)]
JG074D_meta = JG074D_meta[JG074D_meta.component == "L2_500_like"]

data = append_data(JG074D_data)
meta = append_meta(JG074D_meta, supermix="lunar_earwig")
figs = append_figs(JG074D_rxns, "lunar_earwig", supermix="lunar_earwig")
figs = append_figs(JG074D_rxns, "cosmic_falcon", supermix="lunar_earwig")

15 matching reactions
The shape of `data` is now 160 x 9
The shape of `meta` is now 204 x 4
The shape of `figs` is now 145 x 3
The shape of `figs` is now 160 x 3


# Fig 4 galactic_gorilla

In [18]:
ARG1_wildtype = "S036.0"
GBP6_wildtype = "S036.5"
TMCC1_wildtype = "S044.13"
PRDM1_wildtype = "S_PRDM1_WTb"

ARG1_competitor = "S036.1b"
GBP6_competitor = "S044.6"
TMCC1_competitor = "S037.01.01b"
PRDM1_competitor = "S057.5.7"

fragment_descriptions[ARG1_wildtype] = "ARG1 natural transcript DNA analogue"
fragment_descriptions[GBP6_wildtype] = "GBP6 natural transcript DNA analogue"
fragment_descriptions[TMCC1_wildtype] = "TMCC1 natural transcript DNA analogue"
fragment_descriptions[PRDM1_wildtype] = "PRDM1 natural transcript DNA analogue"
fragment_descriptions[ARG1_competitor] = "ARG1 competitor"
fragment_descriptions[GBP6_competitor] = "GBP6 competitor"
fragment_descriptions[TMCC1_competitor] = "TMCC1 competitor"
fragment_descriptions[PRDM1_competitor] = "PRDM1 competitor"

ARG1_comp_lg10_copies = 1.4
GBP6_comp_lg10_copies = 1.0
TMCC1_comp_lg10_copies = 1.8
PRDM1_comp_lg10_copies = 3.8

ARG1_p1 = "FP002"
ARG1_p2 = "FP002"
GBP6_p1 = "FP001"
GBP6_p2 = "RP001"
TMCC1_p1 = "FP004"
TMCC1_p2 = "RP004"
PRDM1_p1 = "FP003"
PRDM1_p2 = "RP008"

ARG1_pnM = 100
GBP6_pnM = 80
TMCC1_pnM = 80
PRDM1_pnM = 40

galactic_gorilla_probes = [
    "Pr001",
    "Pr002",
    "Pr003",
    "Pr004",
    "Pr005",
    "Pr007",
    "Pr008",
    "Pr009",
]

In [19]:
for frag, name in zip(
    [
        ARG1_wildtype,
        GBP6_wildtype,
        TMCC1_wildtype,
        PRDM1_wildtype,
        ARG1_competitor,
        GBP6_competitor,
        TMCC1_competitor,
        PRDM1_competitor,
    ],
    [
        "ARG1_wildtype",
        "GBP6_wildtype",
        "TMCC1_wildtype",
        "PRDM1_wildtype",
        "ARG1_competitor",
        "GBP6_competitor",
        "TMCC1_competitor",
        "PRDM1_competitor",
    ],
):
    print(f"Fragment: {frag} ({name})")
    try:
        f = lib.get_by_name(Fragment, frag)
    except Exception as e:
        print(f"Fragment {frag} not found in LIMS")
        continue

    print(f"Probes: {f.amplicon.core.probe.name}")
    print()

Fragment: S036.0 (ARG1_wildtype)
Probes: Pr004

Fragment: S036.5 (GBP6_wildtype)
Probes: Pr001

Fragment: S044.13 (TMCC1_wildtype)
Probes: Pr002

Fragment: S_PRDM1_WTb (PRDM1_wildtype)
Probes: Pr005

Fragment: S036.1b (ARG1_competitor)
Probes: Pr008

Fragment: S044.6 (GBP6_competitor)
Probes: Pr003

Fragment: S037.01.01b (TMCC1_competitor)
Probes: Pr007

Fragment: S057.5.7 (PRDM1_competitor)
Fragment S057.5.7 not found in LIMS


In [20]:
galactic_gorilla_smix = pd.DataFrame(
    data=[
        [ARG1_competitor, int(10**ARG1_comp_lg10_copies), "copies"],
        [GBP6_competitor, int(10**GBP6_comp_lg10_copies), "copies"],
        [TMCC1_competitor, int(10**TMCC1_comp_lg10_copies), "copies"],
        [PRDM1_competitor, int(10**PRDM1_comp_lg10_copies), "copies"],
        [ARG1_p1, ARG1_pnM, "nM"],
        [ARG1_p2, ARG1_pnM, "nM"],
        [GBP6_p1, GBP6_pnM, "nM"],
        [GBP6_p2, GBP6_pnM, "nM"],
        [TMCC1_p1, TMCC1_pnM, "nM"],
        [TMCC1_p2, TMCC1_pnM, "nM"],
        [PRDM1_p1, PRDM1_pnM, "nM"],
        [PRDM1_p2, PRDM1_pnM, "nM"],
    ]
    + [[p, 200, "nM"] for p in galactic_gorilla_probes],
    columns=["component", "concentration", "units"],
)

smix = append_smix(galactic_gorilla_smix, "galactic_gorilla")

The shape of `smix` is now 61 x 4


In [21]:
JG069J_data = process_experiment("JG069J", "JG069J Final TB Experiment.xlsx").drop(
    columns=["Target", "Copies", "Sample"]
)

JG069J_meta = (
    pd.read_csv(data_pth / "JG069J Plate Map.csv")[
        ["PRDM1", "TMCC1", "GBP6", "ARG1", "Well"]
    ]
    .melt(id_vars=["Well"], var_name="component", value_name="log10_copies")
    .replace(
        {
            "component": {
                "PRDM1": PRDM1_wildtype,
                "TMCC1": TMCC1_wildtype,
                "GBP6": GBP6_wildtype,
                "ARG1": ARG1_wildtype,
            }
        }
    )
    .assign(
        concentration=lambda df: (10 ** df.log10_copies).astype(int),
        units="copies",
        Reaction=lambda df: df.apply(lambda row: f"JG069J_{row.Well:03d}", axis=1),
    )
    .drop(columns=["log10_copies", "Well"])
)


JG069J_rxns = pd.DataFrame(JG069J_data.Reaction.unique(), columns=["Reaction"])

meta = append_meta(JG069J_meta, supermix="galactic_gorilla")
data = append_data(JG069J_data)
figs = append_figs(JG069J_rxns, "galactic_gorilla", supermix="galactic_gorilla")

The shape of `meta` is now 2004 x 4
The shape of `data` is now 864 x 9
The shape of `figs` is now 512 x 3


In [22]:
S057_5_7_seq = "GCATCAACCTGTACTCATCTTACGTCACCCTATTCCTTTCATTGTTTCTAGCACCGTCTTAATCTACACCGCACTGCCCAGGAAGAACTATCGAGAAAGCGCCGTAGTGTTGTGATTTGTTGGGTGGAGAAAAGCCACATGAATGCAAAATAAATACTAAACAAAACTAACAACACAAAAATGGAACACGGCCTTTGACAGTGATAAACTGGAGAGAAACCATACCAATGGGTCGTGGGCTATAACTTTCCTACTTACATTACGGATTGAAACGGGATGGGTCTGTTGGTGCTCGAATACAGATA"
L2_500_seq = "TTTGGTTTGGCTTATGTGGGTTCGGGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGGCCATCAACAGTGGAGCAGTGCCTTGTCTGGAGAATGTATTGAGGGCATACGCAGCACGACCATGCACGTAATCCTAATTAACACCCGCAGATATTTGGGATAGCGGCGTTGTCTGTTTTACGGACGTAAGCATGACACAATTCGACCGGCAACGTGGCTGAACCAATGTGAGAGAATAAATGTGCGTGTGCAAATTACTAATCAAGAAAAGTAGCAGGGAGAGGTATACGGCTCACTTCTATAGCCCATCGCGGACTTATCCGACGCTATATAGGGGAACGCACTACCAGCTTCCAGGTACGTACTATCTTCTATGGACAACCGCCTAGTTCTTAACACTCCAGGAGGCAACTTCAGTACATAGTGGTACGCGCTACGGCGGATGCTTATTTCGATGAGCCTGACAATTAACGGGCAATCAAAGTCGCTTGTTGAAGTAATACATGCGGTGGAGTCTGTCCTGCCAGATCGAGGTCAACAAGAGTTTATATGGCCATTGAATGCCTTGAATGTTCACGCTTAAAGACGCTCCCAACGCCAAGTCCAGAACCATAGGGATTATTGGAGCTCCTTTCTCAAAGGGACAGCCACGAGGAGGGGTGGAAGAAGGCCCTACAGTATTGAGAAAGGCTGGTCTGCTTGAGAAACTTAAAGAACAAGAGTGTGATGTGAAGGATTATGGGGATAGTTTGAGTTCGATCCAGACAGCC"
L2_500_like_seq = "TTTGGTTTGGCTTATGTGGGTTCGGGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGGCCATCAACAGTGGAGCAGTGCCTTGTCTGGAGAATGTATTGAGGGCATACGCAGCACGACCATGCACGTAATCCTAATTAACACCCGCAGATATTTGGGATAGCGGCGTTGTCTGTTTTACGGACGTAAGCATGACACAATTCGACCGGCAACGTGGCTGAACCAATGTGAGAGAATAAATGTGCGTGTGCAAATTACTAATCAAGAAAAGTAGCAGGGAGAGGTATACGGCTCACTTCTATAGCCCATCGCGGACTTATCCGACGCTATATAGGGGAACGCACTACCAGCTTCCAGGTACGTACTATCTTCTATGGACAACCGCCTAGTTCTTAACACTCCAGGAGGCAACTTCAGTACATAGTGGTACGCGCTACGGCGGATGCTTATTTCGATGAGCCTGACAATTAACGGGCAATCAAAGTCGCTTGTTGAAGTAATACATGCGGTGGAGTCTGTCCTGCCAGATCGAGGTCAACAAGAGTTTATATGGCCATTGAATGCCTTGAATGTTCACGCTTAAAGACGCTCCCAACGCCAAGTCCAGAACCATAGGGATTATTGGAGCTCCTTTCTCAAAGGGACAGCCACGAGGAGGGGTGGAAGAAGGCCCTACAGTATTTACAGGCTGGGCCAAATGTCAGAAAATTAAAGAACAAGAGTGTGATGTGAAGGATTATGGGGATAGTTTGAGTTCGATCCAGACAGCC"

fragment_sequences["S057.5.7"] = S057_5_7_seq
fragment_sequences["L2_500"] = L2_500_seq
fragment_sequences["L2_500_like"] = L2_500_like_seq

In [23]:
FP003 = lib.get_by_name(Primer, "FP003")
RP008 = lib.get_by_name(Primer, "RP008")
Pr009 = lib.get_by_name(ProbeSeq, "Pr009")

A057_5_7_start = S057_5_7_seq.find(FP003.sequence)
C057_5_7_start = A057_5_7_start + len(FP003.sequence)
C057_5_7_end = S057_5_7_seq.find(RP008.reverse_complement)
A057_5_7_end = C057_5_7_end + len(RP008.reverse_complement)

C057_5_7_seq = S057_5_7_seq[C057_5_7_start:C057_5_7_end]
A057_5_7_seq = S057_5_7_seq[A057_5_7_start:A057_5_7_end]

# C057_5_7 = Core(
#     name="C057.5.7",
#     sequence=C057_5_7_seq,
#     probe=Pr009,
# )
# A057_5_7 = Amplicon(
#     name="A057.5.7",
#     sequence=A057_5_7_seq,
#     core=C057_5_7,
#     primers=[FP003, RP008],
# )
# S057_5_7 = Fragment(
#     name="S057.5.7",
#     amplicons=[A057_5_7],
#     sequence=S057_5_7_seq,
# )

# Fig 5 happy_jaguar

In [24]:
emri_comp_name = "S073.1.5"
ifi44L_comp_name = "S073.0.2"
ifi44L_wt_name = "S_IFI44L_WT"
emri_wt_name = "S_EMR1_ADGRE1_WT"

fragment_descriptions[emri_comp_name] = "EMR1 competitor"
fragment_descriptions[ifi44L_comp_name] = "IFI44L competitor"
fragment_descriptions[ifi44L_wt_name] = "IFI44L natural transcript DNA analogue"
fragment_descriptions[emri_wt_name] = "EMR1 natural transcript DNA analogue"

happy_jaguar_smix = pd.DataFrame(
    [
        ["S073.1.5", 10**3.6, "copies"],  # EMRI competitor
        ["FP014", 60, "nM"],  # EMRI primer
        ["RP014", 60, "nM"],  # EMRI primer
        ["Pr001", 200, "nM"],  # EMRI competitive probe
        ["Pr007", 200, "nM"],
        ["S073.0.2", 10**4.5, "copies"],  # IFI44L competitor
        ["FP017", 70, "nM"],  # IFI44L primer
        ["RP015", 70, "nM"],  # IFI44L primer
        ["Pr004", 200, "nM"],  # IFI44L competitive probe
        ["Pr008", 200, "nM"],
        ["Taqman Fast Virus Master Mix", 1, "X"],
    ],
    columns=["component", "concentration", "units"],
)
smix = append_smix(happy_jaguar_smix, "happy_jaguar")

The shape of `smix` is now 72 x 4


In [25]:
JG073N_data = process_experiment("JG073N", "JG073N Febrile Signature Gen2.xlsx")

JG073_endpoints = pd.read_csv(data_pth / "JG073N Endpoints.csv").rename(
    columns={"SignalDifference": "FAM-HEX"}
)

JG073N_data = JG073N_data.merge(
    JG073_endpoints[
        ["Experiment", "Well", "IFI44L Copies", "EMRI Copies", "Material", "ng RNA"]
    ]
)

JG073N_data = JG073N_data[
    (JG073N_data["IFI44L Copies"] > 0)
    & (JG073N_data["EMRI Copies"] > 0)
    & (JG073N_data["Material"] == "RNA")
    & (JG073N_data["ng RNA"] == 1)
]

JG073N_meta = (
    JG073N_data[["Reaction", "IFI44L Copies", "EMRI Copies"]]
    .melt(id_vars=["Reaction"], var_name="component", value_name="concentration")
    .assign(
        units="copies",
        concentration=lambda df: (10**df.concentration).astype(int),
    )
    .replace(
        {
            "component": {
                "IFI44L Copies": ifi44L_wt_name,
                "EMRI Copies": emri_wt_name,
            }
        }
    )
)

JG073N_rxns = JG073N_meta.Reaction.unique()

data = append_data(JG073N_data)
meta = append_meta(JG073N_meta, supermix="happy_jaguar")
figs = append_figs(JG073N_rxns, "angry_lemur", supermix="happy_jaguar")
figs = append_figs(JG073N_rxns, "excited_manatee", supermix="happy_jaguar")

The shape of `data` is now 962 x 9
The shape of `meta` is now 2151 x 4
The shape of `figs` is now 561 x 3
The shape of `figs` is now 610 x 3


# Fig ED1 shy_quokka

In [26]:
# order_3648138 = {
#     # Order Date 4/21/2022
#     'S075_WT':
#     'S075_SNV':
# }

# order_3650461 = {
#     # Order Date 4/26/2022
#     'EGFR L858 Blocker C3':  'GATCACAGATTTTGGGCTGGCCAAACATAA/3SpC3/',
#     'EGFR L858 Blocker DxxDM': ' GATCACAGATTTTGGGCTGGCCAAAC/iSpC3/iSpC3/CA'
# }

order_3399189 = {
    # Order Date 3/2/2021
    "EGFR LNA Blocker C3": "GATCACAGATTTTGGGC+TGGCCAAACATAA/3SpC3/"
}

order_3186940 = {
    # Order Date 2/5/2020
    "EGFR L858 p0": "GCAGCATGTCAAGATCACAGATT",
    "EGFR L858 p1": "ACCTCCTTACTTTGCCTCCTTC",
    "EGFR L858 p2": "CATTTGCTTCAACAGTGACTACG",
    "EGFR L858 p3": "GCTATTGCTGGGATTTTGAGG",
    # 'EGFR L858 Blocker C3': 'GATCACAGATTTTGGGCTGGCCAAACATAA/3SpC3/'
}

In [27]:
fragment_sequences["S075_SNV"] = (
    "GTTGAAGTGGTAGGGATTGCGTTTGTGTGTTCTGCTGTTCATGTGTCGTGCTTCTTTCCTCTAGTTTCCGTCATTGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGAGCAGTTTGGCCCGCCCAAAATCTGTGATCTTGACATGCTGCGATTCGTAACTGGACCGTACTAAAGCAGGATTCAGATTATTGGCCGTTCGTTCTTGTTAGCTCCGTCATTCAAATACCTTACCACATATCCAACCAAAGAGGCAAGTCTCATCAAAGCCCTCCCG"
)
fragment_sequences["S075_WT"] = (
    "GTTGAAGTGGTAGGGATTGCGTTTGTGTGTTCTGCTGTTCATGTGTCGTGCTTCTTTCCTCTAGTTTCCGTCATTGCAAGAACCAAGACCCTCAGGGAGGGAATCACAGTCACTGGGAATCGTCTGGGAACTCTGGCAGTGACTTATGTAGAGAGCAGTTTGGCCAGCCCAAAATCTGTGATCTTGACATGCTGCGATTCGTAACTGGACCGTACTAAAGCAGGATTCAGATTATTGGCCGTTCGTTCTTGTTAGCTCCGTCATTCAAATACCTTACCACATATCCAACCAAAGAGGCAAGTCTCATCAAAGCCCTCCCG"
)
primer_sequences["EGFR L858R Blocker"] = "GATCACAGATTTTGGGC+TGGCCAAACATAA/3SpC3/"

fragment_descriptions["S075_SNV"] = "DNA sequence with EGFR L858R SNV"
fragment_descriptions["S075_WT"] = "DNA sequence with EGFR L858R WT"

proud_robin_smix = pd.DataFrame(
    [["FP0013", 100, "nM"], ["FP001", 100, "nM"], ["EvaGreen", 1, "X"]],
    columns=["component", "concentration", "units"],
)
smix = append_smix(proud_robin_smix, "proud_robin")

humble_scorpion_smix = pd.DataFrame(
    [
        ["FP0013", 100, "nM"],
        ["FP001", 100, "nM"],
        ["FP005", 100, "nM"],
        ["FP005", 100, "nM"],
        ["Pr002", 200, "nM"],
        ["Pr003", 200, "nM"],
        ["S057.3.2", 100, "copies"],
        ["S057.4.2", 100, "copies"],
    ],
    columns=["component", "concentration", "units"],
)
smix = append_smix(humble_scorpion_smix, "humble_scorpion")

The shape of `smix` is now 76 x 4
The shape of `smix` is now 84 x 4


In [28]:
JG075A_data = process_experiment("JG075A", "JG075A EGFR Blocker Stoichiometry.xlsx")

# Denote reaction conditions
with np.errstate(divide="ignore"):
    JG075A_data = (
        JG075A_data.merge(pd.read_csv(data_pth / "JG075A Plate Map.csv"))
        .assign(lg10_Blocker=lambda df: np.log10(df["Blocker μM"]))
        .replace({"lg10_Blocker": {-np.inf: -2}})
        .query("Well != 73")  # Outlier, weird drift in baseline
        .query("Copies >= 10**4")
        .query("Blocker == 'L-MMMMx'")
    )

frags = (
    JG075A_data[["Reaction", "Target", "Copies"]]
    .rename(
        columns={
            "Target": "component",
            "Copies": "concentration",
        }
    )
    .assign(units="copies")
)

blocker = (
    JG075A_data[["Reaction", "Blocker μM"]]
    .rename(
        columns={
            "Blocker μM": "concentration",
        }
    )
    .assign(component="EGFR L858R Blocker", units="μM")
)

JG075A_meta = pd.concat([frags, blocker]).sort_values("Reaction")
JG075A_rxns = JG075A_meta.Reaction.unique()

data = append_data(JG075A_data)
meta = append_meta(JG075A_meta, supermix="proud_robin")
figs = append_figs(JG075A_rxns, "proud_robin", supermix="proud_robin")
figs = append_figs(JG075A_rxns, "sour_dolphin", supermix="proud_robin")

The shape of `data` is now 1032 x 9
The shape of `meta` is now 2361 x 4
The shape of `figs` is now 680 x 3
The shape of `figs` is now 750 x 3


In [29]:
JG075B_data = process_experiment(
    "JG075B", "JG075B L-MMMMx blocker tripartite competition separate.xlsx"
)

with np.errstate(divide="ignore"):
    # Denote reaction conditions
    JG075B_data = (
        JG075B_data.replace(
            {
                "Target": {
                    "S075_WTFAM": "S075_WT",
                    "S075_WTHEX": "S075_WT",
                    "S075_SNVFAM": "S075_SNV",
                    "S075_SNVHEX": "S075_SNV",
                }
            }
        )
        .merge(pd.read_csv(data_pth / "JG075B Plate Map.csv"))
        .assign(lg10_Blocker=lambda df: np.log10(df["Blocker μM"]))
        .replace({"lg10_Blocker": {-np.inf: -2}})
        .query('Configuration == "2+2"')
        .query('Blocker == "L-MMMMx"')
    )

JG075B_data = JG075B_data[(JG075B_data.lg10_Blocker % 0.5).isin([0.0, np.nan])]

frags = (
    JG075B_data[["Reaction", "Target", "Copies"]]
    .rename(
        columns={
            "Target": "component",
            "Copies": "concentration",
        }
    )
    .assign(units="copies")
)

blocker = (
    JG075B_data[["Reaction", "Blocker μM"]]
    .rename(
        columns={
            "Blocker μM": "concentration",
        }
    )
    .assign(component="EGFR L858R Blocker", units="μM")
)

JG075B_meta = pd.concat([frags, blocker]).sort_values("Reaction")
JG075B_rxns = JG075B_meta.Reaction.unique()

data = append_data(JG075B_data)
meta = append_meta(JG075B_meta, supermix="humble_scorpion")
figs = append_figs(JG075B_rxns, "humble_scorpion", supermix="humble_scorpion")
figs = append_figs(JG075B_rxns, "strong_xerus", supermix="humble_scorpion")

The shape of `data` is now 1182 x 9
The shape of `meta` is now 2586 x 4
The shape of `figs` is now 825 x 3
The shape of `figs` is now 900 x 3


In [30]:
JG075F_data = process_experiment(
    "JG075F", "JG075F L-MMMMx blocker tripartite competition matrix.xlsx"
)

JG075F_data = JG075F_data.drop(columns=["Sample"]).merge(
    pd.read_csv(data_pth / "JG075F Plate Map.csv"), on="Well"
)

frags = (
    JG075F_data[["Reaction", "WT_lg10_Copies", "SNV_lg10_Copies"]]
    .melt(
        id_vars=["Reaction"],
        value_vars=["WT_lg10_Copies", "SNV_lg10_Copies"],
        var_name="Target",
        value_name="lg10_Copies",
    )
    .assign(
        Copies=lambda df: (10.0 ** df.lg10_Copies).astype(int),
        Target=lambda df: 'S075_' + df.Target.str.removesuffix('_lg10_Copies'),
    )
    .rename(
        columns={
            "Target": "component",
            "Copies": "concentration",
        }
    )
    .assign(units="copies")
)

blocker = (
    JG075F_data[["Reaction", "Blocker μM"]]
    .rename(
        columns={
            "Blocker μM": "concentration",
        }
    )
    .assign(component="EGFR L858R Blocker", units="μM")
)

JG075F_meta = pd.concat([frags, blocker]).sort_values("Reaction")
JG075F_rxns = JG075F_meta.Reaction.unique()

data = append_data(JG075F_data)
meta = append_meta(JG075F_meta, supermix="humble_scorpion")
figs = append_figs(JG075F_rxns, "noisy_tapir", supermix="humble_scorpion")

The shape of `data` is now 1928 x 9
The shape of `meta` is now 4078 x 4
The shape of `figs` is now 1273 x 3


# Fig S2 sharp_jellyfish

In [31]:
JG063A_data = process_experiment(
    "JG063A", "JG063A Competitor Concentration Sweep.xlsx"
).assign(Target=lambda df: df.Target.str.split("_").str[0])
JG063B_data = process_experiment(
    "JG063B", "JG063B Competitor concentration sweep 2.xlsx"
).assign(Target=lambda df: df.Target.str.split("_").str[0])
JG063I_data = process_experiment("JG063I", "JG063I Primer Probe Sweep 5 logs.xlsx")
JG034_data = process_experiment(
    "JG034", "JG034 TMCC1 Gen2 Competitors - 59C v3.xlsx"
).assign(Target=lambda df: df.Target.str.split(" - ").str[0])

for qs in [
    JG063A_data,
    JG063B_data,
    JG063I_data,
    JG034_data,
]:
    qs.drop(qs.index[qs.Well == 281], inplace=True)

JG063I_primerprobe = (
    JG063I_data.Sample.str.removesuffix(" primer+probe")
    .str.split("+", expand=True)
    .rename(columns={0: "Primer nM", 1: "Probe nM"})
    .astype(int)
)

JG063I_data = JG063I_data.merge(JG063I_primerprobe, left_index=True, right_index=True)
JG063I_data = JG063I_data[JG063I_data["Probe nM"] == 200]

In [32]:
primers_063 = [
    ["FP004", 100, "nM"],
    ["RP004x", 100, "nM"],
]

probes_063 = [
    ["Pr002", 200, "nM"],
    ["Pr003", 200, "nM"],
]

primers_034 = [
    ["FP004", 125, "nM"],
    ["RP004", 125, "nM"],
]

probes_034 = [
    ["Pr002", 250, "nM"],
    ["Pr003", 250, "nM"],
]

dull_koala_smix = pd.DataFrame(
    probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)
smix = append_smix(dull_koala_smix, "dull_koala")

bright_lizard_i_smix = pd.DataFrame(
    [["S056.4.3", 10**6, "copies"]] + probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_i_smix, "bright_lizard_i")

bright_lizard_ii_smix = pd.DataFrame(
    [["S056.4.1", 10**4, "copies"]] + probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_ii_smix, "bright_lizard_ii")

bright_lizard_iii_smix = pd.DataFrame(
    [[seq_aliases["GC55"], 10**5, "copies"]] + probes_034 + primers_034,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_iii_smix, "bright_lizard_iii")

bright_lizard_iv_smix = pd.DataFrame(
    [["S056.4.4", 10**5, "copies"]] + probes_063 + primers_063,
    columns=["component", "concentration", "units"],
)  # wt 'S056.4.12'
smix = append_smix(bright_lizard_iv_smix, "bright_lizard_iv")

bright_lizard_v_smix = pd.DataFrame(
    [[seq_aliases["GC15"], 10**5, "copies"]] + probes_034 + primers_034,
    columns=["component", "concentration", "units"],
)
smix = append_smix(bright_lizard_v_smix, "bright_lizard_v")

The shape of `smix` is now 89 x 4
The shape of `smix` is now 95 x 4
The shape of `smix` is now 101 x 4
The shape of `smix` is now 107 x 4
The shape of `smix` is now 113 x 4
The shape of `smix` is now 119 x 4


In [33]:
dull_koala_data = JG063A_data.query('Sample == "S056.4.2_vs_S056.4.12"')
dull_koala_meta = extract_meta(dull_koala_data)


data = append_data(dull_koala_data)
meta = append_meta(dull_koala_meta, supermix="dull_koala")
figs = append_figs(dull_koala_data.Reaction.unique(), "dull_koala", supermix="dull_koala")

The shape of `data` is now 2088 x 9
The shape of `meta` is now 4318 x 4
The shape of `figs` is now 1353 x 3


In [34]:
bright_lizard_data = pd.concat(
    [
        JG063A_data,
        JG063B_data,
        JG034_data,
    ]
)


def bright_lizard_filter(grp):
    conditions = [
        (grp.Experiment == "JG063A")
        & (grp.Reporter == "HEX")
        & (grp.Copies == 10**5)
        & (grp.Sample == "S056.4.4_vs_S056.4.12"),
        (grp.Experiment == "JG063B")
        & (grp.Reporter == "HEX")
        & (grp.Copies == 10**4)
        & (grp.Sample == "S056.4.1_vs_S056.4.12"),
        (grp.Experiment == "JG063B")
        & (grp.Reporter == "HEX")
        & (grp.Copies == 10**6)
        & (grp.Sample == "S056.4.3_vs_S056.4.12"),
        (grp.Experiment == "JG034") & (grp.Reporter == "HEX"),
    ]

    bright_lizard_targets = ["S056.4.3", "S056.4.1", "GC55", "S056.4.4", "GC15"]
    must_have = (grp.Reporter == "FAM") & (grp.Copies > 10**1)

    return (
        reduce(or_, [cond.any() for cond in conditions])
        & must_have.any()
        & grp.Target.isin(bright_lizard_targets).any()
    )


bright_lizard_rxns = filter_reactions(bright_lizard_data, bright_lizard_filter)

bright_lizard_data = bright_lizard_data[
    bright_lizard_data.Reaction.isin(bright_lizard_rxns)
]
# bright_lizard_i_data = bright_lizard_data.query("Target == 'S056.4.3'").assign(
#     Target="S056.4.12"
# )
# bright_lizard_ii_data = bright_lizard_data.query("Target == 'S056.4.1'").assign(
#     Target="S056.4.12"
# )
# bright_lizard_iii_data = bright_lizard_data.query("Target == 'GC55'").assign(
#     Target="S044.13"
# )
# bright_lizard_iv_data = bright_lizard_data.query("Target == 'S056.4.4'").assign(
#     Target="S056.4.12"
# )
# bright_lizard_v_data = bright_lizard_data.query("Target == 'GC15'").assign(
#     Target="S044.13"
# )

bright_lizard_i_data = bright_lizard_data.query("Sample == 'S056.4.3_vs_S056.4.12'")
bright_lizard_ii_data = bright_lizard_data.query("Sample == 'S056.4.1_vs_S056.4.12'")
bright_lizard_iii_data = bright_lizard_data.query("Target == 'GC55'").assign(
    Target="S044.13"
)
bright_lizard_iv_data = bright_lizard_data.query("Sample == 'S056.4.4_vs_S056.4.12'")
bright_lizard_v_data = bright_lizard_data.query("Target == 'GC15'").assign(
    Target="S044.13"
)

# bright_lizard_data.loc[bright_lizard_data.Experiment == 'JG034', 'Target'] = 'S044.13'
bright_lizard_i_meta = extract_meta(bright_lizard_i_data)
bright_lizard_i_meta = bright_lizard_i_meta[~bright_lizard_i_meta.component.isin(bright_lizard_i_smix.component)]
bright_lizard_ii_meta = extract_meta(bright_lizard_ii_data)
bright_lizard_ii_meta = bright_lizard_ii_meta[~bright_lizard_ii_meta.component.isin(bright_lizard_ii_smix.component)]
bright_lizard_iii_meta = extract_meta(bright_lizard_iii_data)
bright_lizard_iii_meta = bright_lizard_iii_meta[~bright_lizard_iii_meta.component.isin(bright_lizard_iii_smix.component)]
bright_lizard_iv_meta = extract_meta(bright_lizard_iv_data)
bright_lizard_iv_meta = bright_lizard_iv_meta[~bright_lizard_iv_meta.component.isin(bright_lizard_iv_smix.component)]
bright_lizard_v_meta = extract_meta(bright_lizard_v_data)
bright_lizard_v_meta = bright_lizard_v_meta[~bright_lizard_v_meta.component.isin(bright_lizard_v_smix.component)]

data = append_data(bright_lizard_i_data)
data = append_data(bright_lizard_ii_data)
data = append_data(bright_lizard_iii_data)
data = append_data(bright_lizard_iv_data)
data = append_data(bright_lizard_v_data)
meta = append_meta(bright_lizard_i_meta, supermix="bright_lizard_i")
meta = append_meta(bright_lizard_ii_meta, supermix="bright_lizard_ii")
meta = append_meta(bright_lizard_iii_meta, supermix="bright_lizard_iii")
meta = append_meta(bright_lizard_iv_meta, supermix="bright_lizard_iv")
meta = append_meta(bright_lizard_v_meta, supermix="bright_lizard_v")
figs = append_figs(bright_lizard_i_meta.Reaction.unique(), "bright_lizard_i", supermix="bright_lizard_i")
figs = append_figs(bright_lizard_ii_meta.Reaction.unique(), "bright_lizard_ii", supermix="bright_lizard_ii")
figs = append_figs(bright_lizard_iii_meta.Reaction.unique(), "bright_lizard_iii", supermix="bright_lizard_iii")
figs = append_figs(bright_lizard_iv_meta.Reaction.unique(), "bright_lizard_iv", supermix="bright_lizard_iv")
figs = append_figs(bright_lizard_v_meta.Reaction.unique(), "bright_lizard_v", supermix="bright_lizard_v")
# figs = append_figs(bright_lizard_data.Reaction.unique(), "bright_lizard")

70 matching reactions
The shape of `data` is now 2116 x 9
The shape of `data` is now 2144 x 9
The shape of `data` is now 2144 x 9
The shape of `data` is now 2172 x 9
The shape of `data` is now 2200 x 9
The shape of `meta` is now 4346 x 4
The shape of `meta` is now 4374 x 4
The shape of `meta` is now 4388 x 4
The shape of `meta` is now 4416 x 4
The shape of `meta` is now 4444 x 4
The shape of `figs` is now 1367 x 3
The shape of `figs` is now 1381 x 3
The shape of `figs` is now 1395 x 3
The shape of `figs` is now 1409 x 3
The shape of `figs` is now 1423 x 3


In [38]:
fragment_descriptions['S056.4.12'] = "Arbitrary sequence similar to TMCC1"
fragment_descriptions['S056.4.3'] = "Arbitrary competitor sequence"
fragment_descriptions['S056.4.1'] = "Arbitrary competitor sequence"
fragment_descriptions['S056.4.4'] = "Arbitrary competitor sequence"
fragment_descriptions['S056.4.2'] = "Arbitrary competitor sequence"
fragment_descriptions['GC55'] = "Arbitrary competitor sequence"
fragment_descriptions['GC15'] = "Arbitrary competitor sequence"

In [39]:
fragment_descriptions["S056.1.1"] = "Arbitrary competitor sequence"
fragment_descriptions["S056.1.12"] = "Arbitrary sequence similar to GBP6"

In [40]:
probes_063I = [
    ["Pr002", 200, "nM"],
    ["Pr003", 200, "nM"],
]

dim_mongoose_smix = pd.DataFrame(
    [["S056.1.1", 10**5, "copies"]] + probes_063I,
    columns=["component", "concentration", "units"],
)

JG063I_primer_nMs = (
    JG063I_data[["Reaction", "Primer nM"]]
    .rename(columns={"Primer nM": "concentration"})
    .assign(units="nM")
    .drop_duplicates()
)
JG063I_primer_nMs = pd.concat(
    [
        JG063I_primer_nMs.assign(component="FP001"),
        JG063I_primer_nMs.assign(component="RP001x"),
    ],
    ignore_index=True,
)


JG063I_meta = extract_meta(JG063I_data).query("component != 'S056.1.1'")
JG063I_meta = pd.concat(
    [JG063I_meta, JG063I_primer_nMs],
    ignore_index=True,
)

JG063I_rxns = JG063I_meta.Reaction.unique()

smix = append_smix(dim_mongoose_smix, "dim_mongoose")
meta = append_meta(JG063I_meta, supermix="dim_mongoose")
data = append_data(JG063I_data)
figs = append_figs(JG063I_rxns, "dim_mongoose", supermix="dim_mongoose")

The shape of `smix` is now 123 x 4
The shape of `meta` is now 4764 x 4
The shape of `data` is now 2360 x 9
The shape of `figs` is now 1503 x 3


# Fig S3 quick_nematode

In [41]:
quick_nematode_smix = pd.DataFrame(
    probes_071b + primers_071b,
    columns=["component", "concentration", "units"],
)
smix = append_smix(quick_nematode_smix, "quick_nematode")

The shape of `smix` is now 130 x 4


In [42]:
JG071B_data = process_experiment("JG071B", "JG071B Tripartite sweep GBP6.xlsx")
JG071B_data = JG071B_data[~JG071B_data.Well.isin([54, 281])]

JG071B_meta = parse_rxn_specs("JG071B Reaction Specifications.json")
JG071B_meta = neaten_meta(JG071B_meta.merge(JG071B_data[["Reaction", "Sample"]]))
JG071B_rxns = JG071B_meta.Reaction.unique()

fragment_descriptions["S057.3.2"] = "GBP6 tripartite competitor 1 (poorly tuned)"
fragment_descriptions["S057.4.2"] = "GBP6 tripartite competitor 2 (poorly tuned)"

data = append_data(JG071B_data)
meta = append_meta(JG071B_meta, supermix="quick_nematode")
figs = append_figs(JG071B_rxns, "sturdy_orca", supermix="quick_nematode")
figs = append_figs(JG071B_rxns, "covert_parrot", supermix="quick_nematode")

The shape of `data` is now 3032 x 9
The shape of `meta` is now 6147 x 4
The shape of `figs` is now 1852 x 3
The shape of `figs` is now 2201 x 3


# Fig S4 rapid_quetzal

In [43]:
primers_071a = [
    ["FP002", 100, "nM"],
    ["RP002", 100, "nM"],
    ["FP006", 100, "nM"],
    ["RP006", 100, "nM"],
]

probes_071a = [
    ["Pr002", 200, "nM"],
    ["Pr003", 200, "nM"],
]

rapid_quetzal_smix = pd.DataFrame(
    probes_071a + primers_071a,
    columns=["component", "concentration", "units"],
)
smix = append_smix(rapid_quetzal_smix, "rapid_quetzal")

The shape of `smix` is now 137 x 4


In [44]:
JG071A_data = process_experiment("JG071A", "JG071A ARG1 Tripartite.xlsx")
JG071A_data = JG071A_data[~JG071A_data.Well.isin([281])]

JG071A_meta = parse_rxn_specs("JG071A Reaction Specifications.json")
JG071A_meta = neaten_meta(JG071A_meta.merge(JG071A_data[["Reaction", "Sample"]]))
JG071A_rxns = JG071A_meta.Reaction.unique()

# Competitor 1: S057.0.1
# Competitor 2: S057.1.2

fragment_descriptions["S057.0.1"] = "ARG1 tripartite competitor 1 (correctly tuned)"
fragment_descriptions["S057.1.2"] = "ARG1 tripartite competitor 2 (correctly tuned)"

data = append_data(JG071A_data)
meta = append_meta(JG071A_meta, supermix="rapid_quetzal")
figs = append_figs(JG071A_rxns, "distant_rhea", supermix="rapid_quetzal")
figs = append_figs(JG071A_rxns, "crimson_stingray", supermix="rapid_quetzal")

The shape of `data` is now 3798 x 9
The shape of `meta` is now 7679 x 4
The shape of `figs` is now 2584 x 3
The shape of `figs` is now 2967 x 3


# Fig S5 urban_tuatara

In [45]:
rural_unicorn_smix = pd.DataFrame(
    probes_074C + primers_074C,
    columns=["component", "concentration", "units"],
)
smix = append_smix(rural_unicorn_smix, "rural_unicorn")

The shape of `smix` is now 146 x 4


In [46]:
JG074C_data = process_experiment("JG074C", "JG074C_L2_500_competitor_sweep.xlsx")

JG074C_meta = parse_rxn_specs("JG074C Reaction Specifications.json")
JG074C_meta = neaten_meta(JG074C_meta.merge(JG074C_data[["Reaction", "Sample"]]))

def this_filter(grp):
    conditions = [
        (grp.component == "S037.01.01b") & (grp.concentration == 10**8),
        (grp.component == "S036.1b") & (grp.concentration == 10**2),
    ]

    return reduce(and_, [~cond.any() for cond in conditions])


JG074C_rxns = filter_reactions(JG074C_meta, this_filter)
JG074C_data = JG074C_data[JG074C_data.Reaction.isin(JG074C_rxns)]
JG074C_meta = JG074C_meta[JG074C_meta.Reaction.isin(JG074C_rxns)]

data = append_data(JG074C_data)
meta = append_meta(JG074C_meta, supermix="rural_unicorn")
figs = append_figs(JG074C_rxns, "rural_unicorn", supermix="rural_unicorn")
figs = append_figs(JG074C_rxns, "coastal_viperfish", supermix="rural_unicorn")

255 matching reactions
The shape of `data` is now 4308 x 9
The shape of `meta` is now 8699 x 4
The shape of `figs` is now 3222 x 3
The shape of `figs` is now 3477 x 3


In [47]:
frozen_weasel_smix = pd.DataFrame(
    probes_074D + primers_074D,
    columns=["component", "concentration", "units"],
)
smix = append_smix(frozen_weasel_smix, "frozen_weasel")

The shape of `smix` is now 155 x 4


In [48]:
JG074D_data = process_experiment("JG074D", "JG074D_L2_500_like_competitor_sweep.xlsx")

JG074D_meta = parse_rxn_specs("JG074D Reaction Specifications.json")
JG074D_meta = neaten_meta(JG074D_meta.merge(JG074D_data[["Reaction", "Sample"]]))
JG074D_rxns = JG074D_meta.Reaction.unique()

data = append_data(JG074D_data)
meta = append_meta(JG074D_meta, supermix="frozen_weasel")
figs = append_figs(JG074D_rxns, "frozen_weasel", supermix="frozen_weasel")
figs = append_figs(JG074D_rxns, "loud_eel", supermix="frozen_weasel")

The shape of `data` is now 5038 x 9
The shape of `meta` is now 10204 x 4
The shape of `figs` is now 3857 x 3
The shape of `figs` is now 4237 x 3


In [49]:
fragment_descriptions["L2_500"] = "Arbitrary natural transcript DNA analogue for antiparallel redundant competitor configuration"
fragment_descriptions["L2_500_like"] = "Arbitrary natural transcript DNA analogue for parallel redundant competitor configuration"

# Fig S6 liquid_yabby

Order number 3650428 on 4/26/2022

In [50]:
liquid_yabby_smix = pd.DataFrame([
    ["FP004", 100, "nM"],
    ["RP004x", 100, "nM"],
],
    columns=["component", "concentration", "units"],
)
smix = append_smix(liquid_yabby_smix, "liquid_yabby")

The shape of `smix` is now 158 x 4


In [51]:
probe_sequences = {
    "Pr003_FAM_Z_IBFQ": "/56-FAM/TAGAGAGGT/ZEN/TACCAGAGCGTTGCC/3IABkFQ/",
    "Pr003_FAM_TAM": "/56-FAM/TAGAGAGGTTACCAGAGCGTTGCC/36-TAMSp/",
    "Pr003_FAM_BHQ1": "/56-FAM/TAGAGAGGTTACCAGAGCGTTGCC/3BHQ_1/",
    "Pr003_TET_Z_IBFQ": "/5TET/TAGAGAGGT/ZEN/TACCAGAGCGTTGCC/3IABkFQ/",
    "Pr003_TET_BHQ1": "/5TET/TAGAGAGGTTACCAGAGCGTTGCC/3BHQ_1/",
    "Pr003_SUN_Z_IBFQ": "/5SUN/TAGAGAGGT/ZEN/TACCAGAGCGTTGCC/3IABkFQ/",
    "Pr003_SUN_IBFQ": "/5SUN/TAGAGAGGTTACCAGAGCGTTGCC/3IABkFQ/",
    "Pr003_HEX_Z_IBFQ": "/5HEX/TAGAGAGGT/ZEN/TACCAGAGCGTTGCC/3IABkFQ/",
    "Pr003_TET_BHQ1": "/5TET/TAGAGAGGTTACCAGAGCGTTGCC/3BHQ_1/", # ordered as Pr003_HEX_BHQ1
    "Pr003_Cy3_IBRQ": "/5Cy3/TAGAGAGGTTACCAGAGCGTTGCC/3IAbRQSp/",
    "Pr003_Cy5_T_IBRQ": "/5Cy5/TAGAGAGGT/TAO/TACCAGAGCGTTGCC/3IAbRQSp/",
    "Pr003_Cy5_IBRQ": "/5Cy5/TAGAGAGGTTACCAGAGCGTTGCC/3IAbRQSp/",
}

In [52]:
JG076EF_paths = {
    "JG076E": pl.Path("JG076E_FP004-RP004x_Set_I.xlsx"),
    "JG076F": pl.Path("JG076F_FP004-RP004x_Set_II.xlsx"),
}

JG076EF_data = pd.concat(
    [
        process_experiment(ex, pth).query('Well != 281')
        for ex, pth in JG076EF_paths.items()
    ]
)

JG076EF_meta = pd.concat(
    [
        name_reactions(
            pd.read_csv(data_pth / f"{ex} Plate Map.csv").assign(Experiment=ex)
        )
        # Well L17 is defective
        .query('Well != 281')
        # The HEX_BHQ1 probe was mistakenly ordered with TET instead of HEX
        .replace({"Probe": {"HEX_BHQ1": "TET_BHQ1"}})
        for ex in JG076EF_paths.keys()
    ]
)

frag_meta = (
    JG076EF_meta[["Reaction", "Target", "lg10_Copies"]]
    .assign(
        Copies = lambda df: 10**df.lg10_Copies
    )
    .drop(columns=['lg10_Copies'])
    .rename(
        columns={
            "Target": "component",
            "Copies": "concentration",
        }
    )
    .assign(units="copies")
)

probe_meta = (
    JG076EF_meta[["Reaction", "Probe"]]
    .assign(
        Probe = lambda df: 'Pr003_' + df.Probe
    )
    .rename(
        columns={
            "Probe": "component",
        }
    )
    .assign(concentration=200, units="nM")
)

JG076EF_meta = pd.concat([frag_meta, probe_meta]).sort_values("Reaction")
JG076EF_rxns = JG076EF_meta.Reaction.unique()

data = append_data(JG076EF_data)
meta = append_meta(JG076EF_meta, supermix="liquid_yabby")
figs = append_figs(JG076EF_rxns, "liquid_yabby", supermix="liquid_yabby")
figs = append_figs(JG076EF_rxns, "liquid_yabby", supermix="liquid_yabby")

The shape of `data` is now 5804 x 9
The shape of `meta` is now 12502 x 4
The shape of `figs` is now 5003 x 3
The shape of `figs` is now 5003 x 3


# Wrap up and export

In [53]:
all_components = set(
    meta.component.unique().tolist() + smix.component.unique().tolist()
)
all_components = [
    c for c in all_components if "mix" not in c.lower() and c != "EvaGreen"
]
primers = [
    c
    for c in all_components
    if c.startswith("FP") or c.startswith("RP") or c == "EGFR L858R Blocker"
]
probes = [c for c in all_components if c.startswith("Pr")]
fragments = [c for c in all_components if c not in primers and c not in probes]

fragment_rows = []
probe_rows = []
primer_rows = []

for name in fragments:
    try:
        seq = lib.get_by_name(Fragment, name).sequence
    except Exception as e:
        try:
            seq = fragment_sequences[name]
        except KeyError as e:
            print(f"Failed to find fragment {name} in LIMS or dictionary")
    finally:
        fragment_rows.append(
            {
                "component": name,
                "sequence": seq,
            }
        )

for name in probes:
    try:
        probe = lib.get_by_name(ProbeSeq, name)
        mods = probe.oligos[0].name.split("_")[1:]
        assert mods[1] == "ZEN"
        assert mods[2] == "IBFQ"
        seq = probe.sequence
        first = seq[:9]
        last = seq[9:]
        fluor = {"HEX": "5HEX", "FAM": "56-FAM"}[mods[0]]
        seq = f"/{fluor}/{first}/ZEN/{last}/3IABkFQ/"
    except Exception as e:
        try:
            seq = probe_sequences[name]
        except KeyError as e:
            print(f"Failed to find probe {name} in LIMS or dictionary")
    finally:
        probe_rows.append(
            {
                "component": name,
                "sequence": seq,
            }
        )

for name in primers:
    try:
        seq = lib.get_by_name(Primer, name).sequence
    except Exception as e:
        try:
            seq = primer_sequences[name]
        except KeyError as e:
            print(f"Failed to find primer {name} in LIMS or dictionary")
    finally:
        primer_rows.append(
            {
                "component": name,
                "sequence": seq,
            }
        )

all_primer_sequences = pd.DataFrame(primer_rows).drop_duplicates()
all_fragment_sequences = pd.DataFrame(fragment_rows).drop_duplicates()
all_fragment_descriptions = pd.DataFrame(
    [
        {"component": key, "description": value}
        for key, value in fragment_descriptions.items()
    ]
)
all_fragment_sequences = all_fragment_sequences.merge(all_fragment_descriptions)

all_probe_sequences = pd.DataFrame(probe_rows).drop_duplicates()

def get_component_type(component):
    if 'mix' in component.lower():
        return "mix"
    if 'evagreen' == component.lower():
        return "evagreen"
    elif component in all_primer_sequences.component.values:
        return "primer"
    elif component in all_fragment_sequences.component.values:
        return "fragment"
    elif component in all_probe_sequences.component.values:
        return "probe"
    else:
        return None

In [54]:
root_pth = here.parent
manuscript_version_path = root_pth / 'MANUSCRIPT_VERSION'
with manuscript_version_path.open('r') as file:
    version = file.read().strip()
    
alias_table = pd.read_csv(root_pth / 'figure_aliases.csv')
alias_to_number = alias_table.set_index('alias')[version].to_dict()

def get_fig_num(fig_alias):
    parts = fig_alias.split('_')
    alias = '_'.join(parts[:2])
    num = alias_to_number[alias] 
    if len(parts) > 2:
        num = f"{num}_{parts[2]}"
    return num

numbered_figs = figs.assign(
    figure = figs.figure_alias.apply(get_fig_num)
).drop(columns='figure_alias')

with pd.ExcelWriter(here / "competitive_reaction_data.xlsx") as writer:
    data.to_excel(writer, sheet_name="reaction_data", index=False)
    meta.to_excel(writer, sheet_name="reaction_metadata", index=False)
    numbered_figs.to_excel(writer, sheet_name="reaction_figures", index=False)
    smix.to_excel(writer, sheet_name="supermix_components", index=False)
    all_fragment_sequences.to_excel(writer, sheet_name="fragment_sequences", index=False)
    all_primer_sequences.to_excel(writer, sheet_name="primer_sequences", index=False)
    all_probe_sequences.to_excel(writer, sheet_name="probe_sequences", index=False)

In [55]:
# grp = numbered_figs[numbered_figs.figure == "S2B"]
# meta.set_index("Reaction").loc[grp.Reaction, :].groupby("Reaction").apply(
#     lambda sgrp: ", ".join(
#         [
#             f"{r.component} @ {r.concentration:.1f}"
#             for _, r in sgrp.sort_values('component').iterrows()
#             # if "supermix" not in r.component
#         ]
#     )
# ).value_counts()#.mean()

In [ ]:
from pandas.testing import assert_frame_equal


def get_fig_number(fig):
    if fig.count("_") > 1:
        parts = fig.split("_")
        fig_number = alias_to_number["_".join(parts[:2])]
        fig_number = f"{fig_number}_{parts[2]}"
    else:
        fig_number = alias_to_number[fig]
    return fig_number


message_rows = []

for (fig, num), grp in (
    figs.assign(fig_number=figs.figure_alias.apply(get_fig_number))
    .sort_values("fig_number")
    .groupby(["figure_alias", "fig_number"])
):
    av_reps = (
        meta.set_index("Reaction")
        .loc[grp.Reaction, :]
        .groupby("Reaction")
        .filter(lambda sgrp: (sgrp.concentration > 0).all())
        .groupby("Reaction")
        .apply(
            lambda sgrp: ", ".join(
                [
                    f"{r.component} @ {r.concentration:.1f}"
                    for _, r in sgrp.sort_values("component").iterrows()
                ]
            )
        )
        .value_counts()
        .mean()
    )

    this_smix = grp.supermix.unique()
    assert len(this_smix) == 1, f"Unexpected number of supermixes: {this_smix}"
    this_smix = this_smix[0]
    this_smix = smix[smix.SuperMix == this_smix].sort_values(
        ["units", "concentration", "component"]
    ).assign(
        type=lambda df: df.component.apply(get_component_type)
    )

    smixs = (
        meta[meta.component.str.contains("supermix")]
        .set_index("Reaction")
        .loc[grp.Reaction, :]
        .component.unique()
    )
    
    var_components = (
        meta
        .set_index("Reaction")
        .loc[grp.Reaction, :]
        .query("~component.str.contains('supermix')", engine='python')
        .component.unique().tolist()
    )
    
    var_components = list_and(var_components)

    # if fig == "bright_lizard":
    #     continue
    # elif len(smixs) == 1:
    #     this_smix = smix[smix.SuperMix == smixs[0]].sort_values(
    #         ["units", "concentration", "component"]
    #     )
    # elif len(smixs) == 2:
    #     smix0 = smix[smix.SuperMix == smixs[0]][["units", "concentration", "component"]].reset_index(drop=True)
    #     smix1 = smix[smix.SuperMix == smixs[1]][["units", "concentration", "component"]].reset_index(drop=True)
    #     assert_frame_equal(smix0, smix1)
    #     this_smix = smix0
    # else:
    #     raise ValueError(f"Unexpected number of supermixes: {smixs}")

    msg = list_and(
        [
            f"{int(row.concentration)} {row.units} {(row.type+' ' if row.type not in ['mix','evagreen', None] else '')}{row.component}"
            for _, row in this_smix.iterrows()
            if row.component != "TaqMan Fast Advanced Master Mix"
        ]
    )
    # messages.append(msg)

    description = {
        1: "without replicates",
        2: "in duplicate",
        3: "in triplicate",
    }[int(np.round(av_reps))]

    if fig.count("_") > 1:
        parts = fig.split("_")
        fig_number = alias_to_number["_".join(parts[:2])]
        fig_number = f"{fig_number}_{parts[2]}"
    else:
        fig_number = alias_to_number[fig]
    # print(f"Figure {fig_number} {fig}: {msg} {description}.")
    
    if fig == "liquid_yabby":
        msg = "150 nM indicated probe, " + msg
        var_components = "indicated sequence"
    
    message_rows.append(
        {
            "figure": fig_number,
            "figure_alias": fig,
            "message": f"Figure {fig_number} {fig}: {msg}; variable {var_components}; {description}.",
        }
    )
message_df = pd.DataFrame(message_rows)
full_message = "\n".join(message_df.sort_values("figure").message.tolist())
print(full_message)

assert '; variable ;' not in full_message

Figure 2 glass_rattlesnake: 1 X EvaGreen, 125 nM primer FP004, and 125 nM primer RP004; variable S044.27, S044.5, S044.7, and S044.26; in duplicate.
Figure 3Ai plastic_yak: 100000 copies S044.14, 125 nM primer FP004, 125 nM primer RP004, 250 nM probe Pr002, and 250 nM probe Pr003; variable S044.13; in duplicate.
Figure 3Aii ceramic_zebu: 100000 copies S044.14, 125 nM primer FP004, 125 nM primer RP004, 250 nM probe Pr002, and 250 nM probe Pr003; variable S044.13; in duplicate.
Figure 3Bi metal_anchovy: 10000 copies fragment S057.4.2, 1000000 copies fragment S057.3.2, 100 nM primer FP001, 100 nM primer FP005, 100 nM primer RP001, 100 nM primer RP005, 200 nM probe Pr002, and 200 nM probe Pr003; variable S036.5, S057.3.2, and S057.4.2; in duplicate.
Figure 3Bii digital_bear: 10000 copies fragment S057.4.2, 1000000 copies fragment S057.3.2, 100 nM primer FP001, 100 nM primer FP005, 100 nM primer RP001, 100 nM primer RP005, 200 nM probe Pr002, and 200 nM probe Pr003; variable S036.5, S057.3.